- LogisticRegression ROC-AUC SCORE:0.6514
- DecisionTreeClassifier ROC-AUC SCORE:  0.6788
- KNeighborsClassifier ROC-AUC SCORE:  0.5348
- VotingClassifier ROC-AUC SCORE:  0.7180
- RandomForestClassifier ROC-AUC SCORE:  0.8571
- AdaBoostClassifier ROC-AUC SCORE:  0.6767
- GradientBoostingClassifier ROC-AUC SCORE:  0.8622
- XGBoostClassifier ROC-AUC SCORE:  0.8487
- LightGBM ROC-AUC SCORE:  0.8558

In [1]:
import pandas as pd
import numpy as np

In [2]:
churn = pd.read_csv('data/churn_ver02.csv')
target = churn.Exited
churn = churn.drop(columns='Exited')

In [8]:
#0.6788
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier # Wrapper-Class
from sklearn.model_selection import GridSearchCV

In [9]:
dt = DecisionTreeClassifier()
lgbm = LGBMClassifier(n_estimators=300, objective='binary',
                      learning_rate=0.1, max_depth=8)

In [16]:
param = {'criterion':['gini','entropy'],
         'max_depth':range(2,51)}
param2 = {'n_estimators': [100, 200, 300],
          'learning_rate' : [0.01, 0.05, 0.1],
          'max_depth': range(6,10)}

In [17]:
grid_dt = GridSearchCV(dt, param_grid=param, 
                      scoring='roc_auc', cv=4,
                      n_jobs=-1, verbose=1)
grid_lgbm = GridSearchCV(lgbm, param_grid=param2, 
                      scoring='roc_auc', cv=4,
                      n_jobs=-1, verbose=1)             

In [18]:
grid_dt.fit(churn, target)
grid_lgbm.fit(churn, target)

Fitting 4 folds for each of 98 candidates, totalling 392 fits
Fitting 4 folds for each of 36 candidates, totalling 144 fits


GridSearchCV(cv=4,
             estimator=LGBMClassifier(max_depth=8, n_estimators=300,
                                      objective='binary'),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.05, 0.1],
                         'max_depth': range(6, 10),
                         'n_estimators': [100, 200, 300]},
             scoring='roc_auc', verbose=1)

In [19]:
grid_dt.best_estimator_

DecisionTreeClassifier(max_depth=5)

In [20]:
grid_lgbm.best_estimator_

LGBMClassifier(learning_rate=0.01, max_depth=9, n_estimators=300,
               objective='binary')

In [21]:
grid_dt

GridSearchCV(cv=4, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(2, 51)},
             scoring='roc_auc', verbose=1)

In [22]:
grid_lgbm

GridSearchCV(cv=4,
             estimator=LGBMClassifier(max_depth=8, n_estimators=300,
                                      objective='binary'),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.05, 0.1],
                         'max_depth': range(6, 10),
                         'n_estimators': [100, 200, 300]},
             scoring='roc_auc', verbose=1)

- Final Ex

In [23]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(churn, target, test_size=0.3, stratify=target)

In [24]:
dt = DecisionTreeClassifier()
param = {'criterion':['gini','entropy'],
         'max_depth':range(2,51)}
grid_dt = GridSearchCV(dt, param_grid=param, 
                      scoring='roc_auc', cv=4,
                      n_jobs=-1, verbose=1)
grid_dt.fit(x_train, y_train)
print(grid_dt.best_estimator_)

Fitting 4 folds for each of 98 candidates, totalling 392 fits
DecisionTreeClassifier(max_depth=6)


In [25]:
#0.6788
pred = grid_dt.predict_proba(x_test)[:,1]
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, pred)

0.8311323928069118

In [26]:
from sklearn.model_selection import RandomizedSearchCV
dt = DecisionTreeClassifier()
param = {'criterion':['gini','entropy'],
         'max_depth':range(2,51)}
random_dt = RandomizedSearchCV(dt, param_distributions=param, scoring='roc_auc', n_iter=60, cv=4, n_jobs=-1, verbose=1)
random_dt.fit(x_train, y_train)
print(random_dt.best_estimator_)

Fitting 4 folds for each of 60 candidates, totalling 240 fits
DecisionTreeClassifier(max_depth=6)


In [27]:
#0.6788
pred = random_dt.predict_proba(x_test)[:,1]
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, pred)

0.8319264029968233